In [1]:
!pip install -U pytorch_tabular[extra]
!pip install -U pytorch_tabular

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 535.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 24.1 MB/s eta 0:00:00
   ━━━━

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%load_ext cudf.pandas
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from pytorch_tabnet.tab_model import TabNetClassifier
import torch
import torch.nn as nn
import torch.optim as optim
import os
import warnings
warnings.filterwarnings(action='ignore')
from sklearn.model_selection import ParameterGrid

from copy import deepcopy
from rich.progress import Progress

from pytorch_tabular import TabularModel
from pytorch_tabular.models import NodeConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig

In [4]:
import duckdb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 데이터 불러오기
train_path = '/content/drive/MyDrive/train.parquet'

con = duckdb.connect()

# A~E Segment 각각 1만개 샘플링
df = con.query(f"""
                (SELECT *
                FROM read_parquet('{train_path}')
                WHERE Segment = 'A'
                ORDER BY random()
                LIMIT 10000)

                UNION ALL

                (SELECT *
                FROM read_parquet('{train_path}')
                WHERE Segment = 'B'
                ORDER BY random()
                LIMIT 10000)

                UNION ALL

                (SELECT *
                FROM read_parquet('{train_path}')
                WHERE Segment = 'C'
                ORDER BY random()
                LIMIT 10000)

                UNION ALL

                (SELECT *
                FROM read_parquet('{train_path}')
                WHERE Segment = 'D'
                ORDER BY random()
                LIMIT 10000)

                UNION ALL

                (SELECT *
                FROM read_parquet('{train_path}')
                WHERE Segment = 'E'
                ORDER BY random()
                LIMIT 10000)
                """).df()

con.close()

# 결과 확인
print(f"샘플링된 데이터 크기: {len(df)}")
print(df['Segment'].value_counts())
df.head()

# 수치형, 범주형 컬럼 구분
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
categorical_cols.remove('Segment')  # 'Segment'는 타겟이므로 제외

# 입력(X), 출력(y) 분리
X = df[numerical_cols + categorical_cols]
y = df['Segment']

# train/validation 분할
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

# 수치형 데이터 스케일링
scaler = StandardScaler()
X_train_num = pd.DataFrame(scaler.fit_transform(X_train[numerical_cols]), columns=numerical_cols, index=X_train.index)
X_val_num = pd.DataFrame(scaler.transform(X_val[numerical_cols]), columns=numerical_cols, index=X_val.index)

# 범주형 데이터 원-핫 인코딩
X_train_cat = pd.get_dummies(X_train[categorical_cols])
X_val_cat = pd.get_dummies(X_val[categorical_cols])

# 훈련 데이터의 열을 기준으로 검증 데이터 재정렬
X_val_cat = X_val_cat.reindex(columns=X_train_cat.columns, fill_value=0)

# 수치형과 범주형 데이터 결합
X_train_processed = pd.concat([X_train_num, X_train_cat], axis=1)
X_val_processed = pd.concat([X_val_num, X_val_cat], axis=1)

# 테스트 데이터 처리
test_data = pd.read_parquet('/content/drive/MyDrive/test.parquet')
X_test_num = pd.DataFrame(scaler.transform(test_data[numerical_cols]), columns=numerical_cols, index=test_data.index)
X_test_cat = pd.get_dummies(test_data[categorical_cols])
X_test_cat = X_test_cat.reindex(columns=X_train_cat.columns, fill_value=0)
X_test_processed = pd.concat([X_test_num, X_test_cat], axis=1)

# 최종 확인
print(f"훈련 데이터 형태: {X_train_processed.shape}")
print(f"검증 데이터 형태: {X_val_processed.shape}")
print(f"테스트 데이터 형태: {X_test_processed.shape}")


IOException: IO Error: No files found that match the pattern "/content/drive/MyDrive/train.parquet"

In [ ]:
# 입력(X), 출력(y) 분리
X_num = df[numerical_cols]
X_cat = df[categorical_cols]
y = df['Segment']

# ★★★ Segment를 숫자로 변환
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)


In [ ]:
cat_index = X_train.select_dtypes(['object']).columns.to_list()
num_index = X_train.select_dtypes(['int64']).columns.to_list()[:-1]
target = ['Segment']

In [ ]:
data_config = DataConfig(target=target,
                         continuous_cols=num_index,
                         categorical_cols=cat_index)

trainer_config = TrainerConfig(batch_size=64,
                               max_epochs=10,
                               accelerator='gpu',
                               early_stopping='valid_loss',
                               early_stopping_mode ='min',
                               early_stopping_patience=8,
                               checkpoints='valid_loss',
                               checkpoints_mode='min',
                               checkpoints_path='Node_Best',
                               load_best=True,
                               seed=21)

# 논문에서는 Quasi-Hyperbolic Adam 사용
optimizer_config = OptimizerConfig(optimizer='Adam',
                                   optimizer_params={'weight_decay': 1e-5},
                                   lr_scheduler=None)

head_config = LinearHeadConfig(layers="", # No additional layer in head, just a mapping layer to output_dim
                               dropout=0.1,
                               initialization="kaiming").__dict__ # Convert to dict to pass to the model config

model_config = NodeConfig(task='classification',
                          head='LinearHead',
                          head_config=head_config,
                          learning_rate=1e-3,
                          num_layers=2,
                          num_trees=1024,
                          depth=6,
                          additional_tree_output_dim=3,
                          choice_function='entmax15',
                          bin_function='entmoid15',
                          input_dropout=0.0,
                          embedding_dropout=0.1)

tabular_model = TabularModel(data_config=data_config,
                             model_config=model_config,
                             optimizer_config=optimizer_config,
                             trainer_config=trainer_config)

INFO:pytorch_tabular.tabular_model:Experiment Tracking is turned off


In [ ]:

# -----------------------------
# 2. 모델 학습
# -----------------------------
# 2-1. XGBoost
xgb_model = xgb.XGBClassifier(objective='multi:softprob', num_class=5, eval_metric='mlogloss')
xgb_model.fit(X_train, y_train)

# 2-2. TabNet
tabnet_model = TabNetClassifier(n_d=64, n_a=64, n_steps=5, verbose=0)
tabnet_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], patience=30)

# 2-3. NODE, DNF-Net, 1D-CNN은 직접 만들거나 사전 구현된 클래스를 임포트해야 함
# 여기선 간단히 MLP(멀티레이어 퍼셉트론)로 대체 (예시용)
class SimpleMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, output_dim=5):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.layers(x)

# NODE 대체용 간단한 모델
node_model = tabular_model(X_train.shape[1])

# DNF-Net 대체용 간단한 모델
dnf_model = SimpleMLP(X_train.shape[1])

# 1D-CNN 대체용 모델 (1D Conv Layer 포함)
class Simple1DCNN(nn.Module):
    def __init__(self, input_dim, output_dim=5):
        super().__init__()
        self.conv = nn.Conv1d(1, 32, kernel_size=3, padding=1)
        self.fc = nn.Linear(32 * input_dim, output_dim)

    def forward(self, x):
        x = x.unsqueeze(1)  # (batch_size, 1, features)
        x = torch.relu(self.conv(x))
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

cnn_model = Simple1DCNN(X_train.shape[1])

# PyTorch 모델 학습 함수
def train_torch_model(model, X_train, y_train, X_val, y_val, epochs=50):
    model = model.cuda()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.CrossEntropyLoss()
    X_train = torch.tensor(X_train, dtype=torch.float32).cuda()
    y_train = torch.tensor(y_train, dtype=torch.long).cuda()
    X_val = torch.tensor(X_val, dtype=torch.float32).cuda()
    y_val = torch.tensor(y_val, dtype=torch.long).cuda()

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        preds = model(X_train)
        loss = loss_fn(preds, y_train)
        loss.backward()
        optimizer.step()

train_torch_model(node_model, X_train, y_train, X_val, y_val)
train_torch_model(dnf_model, X_train, y_train, X_val, y_val)
train_torch_model(cnn_model, X_train, y_train, X_val, y_val)

# -----------------------------
# 3. Validation 예측 및 가중치 계산
# -----------------------------
val_preds = {}

val_preds['xgb'] = xgb_model.predict_proba(X_val)
val_preds['tabnet'] = tabnet_model.predict_proba(X_val)

node_model.eval()
dnf_model.eval()
cnn_model.eval()

with torch.no_grad():
    val_preds['node'] = torch.softmax(node_model(torch.tensor(X_val, dtype=torch.float32).cuda()), dim=1).cpu().numpy()
    val_preds['dnf'] = torch.softmax(dnf_model(torch.tensor(X_val, dtype=torch.float32).cuda()), dim=1).cpu().numpy()
    val_preds['cnn'] = torch.softmax(cnn_model(torch.tensor(X_val, dtype=torch.float32).cuda()), dim=1).cpu().numpy()

# F1 score 기반 weight 계산
def calc_weight(pred, y_true):
    pred_labels = np.argmax(pred, axis=1)
    return f1_score(y_true, pred_labels, average='macro')

weights = {}
for name in val_preds:
    weights[name] = calc_weight(val_preds[name], y_val)

# 정규화
total = sum(weights.values())
for name in weights:
    weights[name] /= total

print("Model Weights:", weights)

# -----------------------------
# 4. Test 데이터 예측 및 가중 평균
# -----------------------------
test_preds = {}
test_preds['xgb'] = xgb_model.predict_proba(X_test)
test_preds['tabnet'] = tabnet_model.predict_proba(X_test)

with torch.no_grad():
    test_preds['node'] = torch.softmax(node_model(torch.tensor(X_test, dtype=torch.float32).cuda()), dim=1).cpu().numpy()
    test_preds['dnf'] = torch.softmax(dnf_model(torch.tensor(X_test, dtype=torch.float32).cuda()), dim=1).cpu().numpy()
    test_preds['cnn'] = torch.softmax(cnn_model(torch.tensor(X_test, dtype=torch.float32).cuda()), dim=1).cpu().numpy()

# 가중 평균
final_pred = np.zeros_like(test_preds['xgb'])
for name in test_preds:
    final_pred += weights[name] * test_preds[name]

final_labels = np.argmax(final_pred, axis=1)

# -----------------------------
# 5. 최종 F1-score
# -----------------------------
final_f1 = f1_score(y_test, final_labels, average='macro')
print("Final Test Macro F1-Score:", final_f1)


ValueError: could not convert string to float: 'TRAIN_139498'